In [96]:
from google.colab import drive
drive.mount('/content/d')

Drive already mounted at /content/d; to attempt to forcibly remount, call drive.mount("/content/d", force_remount=True).


In [97]:
import pandas as pd
import numpy as np

# Reading the  data



In [98]:
# Reading the meta data
# importing libraries

import gzip
import shutil

# #Path to the meta data zip file. 'sentiment_analysis' is the folder name under 'My Drive'
path1 = '/content/d/MyDrive/CAPSTONE_DATA/meta_Cell_Phones_and_Accessories.json.gz'



# # Path to meta data .json file
path2 = '/content/d/MyDrive/CAPSTONE_DATA/meta_Cell_Phones_and_Accessories.json'

# Unzipping the meta data file
with gzip.open(path1, 'rb') as f_in:
    with open(path2, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# # Reading the unzipped meta data into a Python list. The result will be a list of dictionaries. 
import json

# # Empty list to store the dictonaries
phonemetadata = []

# # Reading the dictionaries in the json file and appending it to the list phonemetadata[]
with open(path2, 'r') as f:
    for line in f:
        phonemetadata.append(json.loads(line))

In [ ]:
# Getting the number of entries in the phonemetadata list
len(phonemetadata)

In [ ]:
#converting the list phonemetadata into a data frame
df_meta=pd.DataFrame(phonemetadata)


In [ ]:
#Reading the .csv file of the phone data into a dataframe

df_phonedata = pd.read_csv('/content/d/MyDrive/CAPSTONE_DATA/Cell_Phones_and_Accessories_5.csv')



In [ ]:
df_phonedata.head(5)

In [ ]:
# pd.read_csv('Cell_Phones_and_Accessories_5.csv', dtype={'overall': 'float', 'verified': 'Bool', 'reviewerID': 'Bool','asin': 'Bool','verified': 'Bool','verified': 'Bool','verified': 'Bool','verified': 'Bool','verified': 'Bool',  })

In [ ]:
len(df_phonedata)

In [ ]:
df_meta.head(2)

In [ ]:
df_meta.shape

In [ ]:
df_meta.drop_duplicates(subset ="asin",keep=False,inplace=True)

In [ ]:
df_meta.shape

# Data pre-processing



In [ ]:
# Converting unix review time to date-time format

#Transforming unixReview time to date time format
from datetime import datetime, timedelta
df_phonedata['Date&Time'] = df_phonedata['unixReviewTime'].apply(lambda d: (datetime.fromtimestamp(d) - timedelta(hours=2)).strftime('%Y-%m-%d'))


In [ ]:
df_phonedata.drop(['unixReviewTime'], axis = 1, inplace=True)

In [ ]:
#joining dfs on 'asin' 
df_f01=df_phonedata.merge(df_meta, left_on='asin', right_on='asin', suffixes=('_data', '_meta'))

In [ ]:
df_f01.head(1)

In [ ]:
from collections import Counter

cat=[]
for i in df_f01['category'].str[2]: 
  cat.append(i)

uni=[]
for k in cat:
  if k not in uni:
    uni.append(k)
print(uni)    


# the two major categories of cell phones are 
#Carrier Cell Phones
#Unlocked Cell Phones. 

In [ ]:
def findstr(x):
  for l in x: 
    if l=='Unlocked Cell Phones' or l=='Carrier Cell Phones': 
      return l 



# EDA

In [ ]:
f1=df_f01['category'].str[2]=='Unlocked Cell Phones' 
f2=df_f01['category'].str[2]=='Carrier Cell Phones' 


In [ ]:
#creating filtered df on above parameters 

dfok=df_f01.loc[f1 | f2]

In [ ]:
len(dfok) 
df_f02=dfok.copy()  #working database for further analysis 

In [ ]:
df_f02.head(2)



In [ ]:
df_f02['style'].value_counts()

In [ ]:
df_f02['style'].isnull().sum()

In [ ]:

df_f02['rank'] = df_f02['rank'].astype(str)

In [ ]:
df_f02['rank']

In [ ]:
#cleaning rank comumn 
df_f02['rnk']=df_f02['rank'].str.extract('([0-9]*,[0-9]*,[0-9]*)', expand=False)


In [ ]:
df_f02.head(2)

In [ ]:
#extracting number of viewed items 
emp=[]
for i in df_f02['also_view']:
  emp.append(len(i))


df_f02['no_of_viewed_items']=emp



In [ ]:
df_f02.no_of_viewed_items

In [ ]:
df_f02.drop(columns=['rank'],inplace=True)

In [ ]:
df_f02.head(1)

In [ ]:
df_meta.head(2)

In [ ]:
#zipping dictionary of asins and products for later analysis 
asins_and_products=dict(zip(df_meta.asin,df_meta.brand))

In [ ]:
df_f02.isnull().sum()/len(df_f02)

In [ ]:
df_f02.drop(columns=['style','vote','image_data','rnk'],inplace=True)

In [ ]:
df_f02.shape

In [ ]:
#cleaning price col
df_f02['price']=df_f02['price'].str.extract('([0-9]{1,}.[0-9]{1,})', expand=False)


In [ ]:
# filling na values of price with 0 

df_f02['price'] = (
    df_f02['price'].fillna(0)
    .astype(float)
    
    .where(df_f02['price'].notnull())
)

In [ ]:
df_f02.drop(columns=['reviewerName','tech1','description','fit','also_buy','also_view','image_meta','tech2','details','date'],inplace=True)

In [ ]:
#cleaning for further analysis 
df_f02.to_csv('clean4.csv')